In [1]:
import math

from lbmpy.session import *
from pystencils.session import *

from lbmpy.moments import MOMENT_SYMBOLS
from collections import OrderedDict

from lbmpy.methods.creationfunctions import create_with_discrete_maxwellian_eq_moments

from lbmpy.phasefield_allen_cahn.parameter_calculation import calculate_parameters_rti
from lbmpy.phasefield_allen_cahn.kernel_equations import *
from lbmpy.phasefield_allen_cahn.force_model import MultiphaseForceModel

In [2]:
try:
    import pycuda
except ImportError:
    pycuda = None
    gpu = False
    target = 'cpu'
    print('No pycuda installed')

if pycuda:
    gpu = True
    target = 'gpu'

In [3]:
stencil_phase = get_stencil("D2Q9")
stencil_hydro = get_stencil("D2Q9")
assert(len(stencil_phase[0]) == len(stencil_hydro[0]))

dimensions = len(stencil_phase[0])

In [4]:
# domain 
L0 = 256
Nx = L0
Ny = 4 * L0
X0 = (Nx/2) + 1
Y0 = (Ny/2) + 1

# time step
tf = 10001
reference_time = 16000

# force acting on the bubble
body_force = [0, 0, 0]
rho_H = 1.0


parameters = calculate_parameters_rti(reference_length=L0,
                                      reference_time=reference_time,
                                      density_heavy=rho_H,
                                      capillary_number=0.26,
                                      reynolds_number=3000,
                                      atwood_number=0.5,
                                      peclet_number=1000,
                                      density_ratio=3,
                                      viscosity_ratio=1)

rho_L = parameters.get("density_light")

mu_H = parameters.get("dynamic_viscosity_heavy")
mu_L = parameters.get("dynamic_viscosity_light")

tau_H = parameters.get("relaxation_time_heavy")
tau_L = parameters.get("relaxation_time_light")

sigma = parameters.get("surface_tension")
M = parameters.get("mobility")
gravitational_acceleration = parameters.get("gravitational_acceleration")


# interface thickness
W = 5
# coeffcient related to surface tension
beta = 12.0 * (sigma/W)
# coeffcient related to surface tension
kappa = 1.5 * sigma*W
# relaxation rate allen cahn (h)
w_c = 1.0/(0.5 + (3.0 * M))


# fields 
domain_size = (Nx, Ny)
dh = ps.create_data_handling((domain_size), periodicity = (True, False), parallel=False, default_target=target)

g = dh.add_array("g", values_per_cell=len(stencil_hydro))
dh.fill("g", 0.0, ghost_layers=True)
h = dh.add_array("h",values_per_cell=len(stencil_phase))
dh.fill("h", 0.0, ghost_layers=True)

g_tmp = dh.add_array("g_tmp", values_per_cell=len(stencil_hydro))
dh.fill("g_tmp", 0.0, ghost_layers=True)
h_tmp = dh.add_array("h_tmp",values_per_cell=len(stencil_phase))
dh.fill("h_tmp", 0.0, ghost_layers=True)

u = dh.add_array("u", values_per_cell=dh.dim)
dh.fill("u", 0.0, ghost_layers=True)

C = dh.add_array("C")
dh.fill("C", 0.0, ghost_layers=True)

force = dh.add_array("force",values_per_cell=dh.dim)
dh.fill("force", 0.0, ghost_layers=True)

In [5]:
tau = 0.5 + tau_L + (C.center) * (tau_H - tau_L)
s8 = 1/(tau)

rho = rho_L + (C.center) * (rho_H - rho_L)

body_force[1] = gravitational_acceleration * rho

In [6]:
method_phase = create_lb_method(stencil=stencil_phase, method='srt', relaxation_rate=w_c, compressible = True)

In [7]:
if len(stencil_hydro) == 9:
    # for D2Q9 a self defined method is used
    x, y, z = MOMENT_SYMBOLS
    moment_table = [sp.Integer(1),
                    -4 + 3*(x**2 + y**2),
                    4 - sp.Rational(21, 2)*(x**2 + y**2) + sp.Rational(9, 2)*(x**2 + y**2)**2,
                    x,
                    (-5 + 3*(x**2 + y**2))*x,
                    y,
                    (-5 + 3*(x**2 + y**2))*y ,
                    x**2 - y**2,
                    x*y]

    relax_table = [1, 1, 1, 1, 1, 1, 1, s8, s8]
    rr_dict = OrderedDict(zip(moment_table, relax_table))
elif len(stencil_hydro) == 19:
    mrt = create_lb_method(method="mrt", stencil=stencil_hydro, relaxation_rates=[1, 1, 1, 1, s8, 1, 1])
    rr_dict = OrderedDict(zip(mrt.moments, mrt.relaxation_rates))
else:
    mrt = create_lb_method(method="mrt", stencil=stencil_hydro, relaxation_rates=[1, 1, s8, 1, 1, 1, 1])
    rr_dict = OrderedDict(zip(mrt.moments, mrt.relaxation_rates))

method_hydro = create_with_discrete_maxwellian_eq_moments(stencil_hydro, rr_dict, compressible=False)

In [8]:
# initialize the domain
def Initialize_distributions():
    
    for block in dh.iterate(ghost_layers=True, inner_ghost_layers=False):
        x, y = block.midpoint_arrays
        y -= 2 * L0
        tmp = 0.1 * Nx * np.cos((2 * math.pi * x) / Nx)
        init_values = 0.5 + 0.5 * np.tanh((y - tmp) / (W / 2))
        block["C"][:, :] = init_values
        
    if gpu:
        dh.all_to_gpu()            
    
    dh.run_kernel(h_init)
    dh.run_kernel(g_init)

In [9]:
h_updates = initializer_kernel_phase_field_lb(h, C, u, method_phase, W)
g_updates = initializer_kernel_hydro_lb(g, u, method_hydro)

h_init = ps.create_kernel(h_updates, target=dh.default_target, cpu_openmp=True).compile()
g_init = ps.create_kernel(g_updates, target=dh.default_target, cpu_openmp=True).compile()

In [10]:
force_h = [f / 3 for f in interface_tracking_force(C, stencil_phase, W)]
force_model_h = MultiphaseForceModel(force=force_h)

force_g = hydrodynamic_force(g, C, method_hydro, tau, rho_H, rho_L, kappa, beta, body_force)
force_model_g = MultiphaseForceModel(force=force_g, rho=rho)

In [11]:
# create the allen cahl lattice boltzmann step for the h field as well as the hydrodynamic
# lattice boltzmann step for the g field
h_tmp_symbol_list = [h_tmp.center(i) for i, _ in enumerate(stencil_phase)]
sum_h = np.sum(h_tmp_symbol_list[:])

method_phase = create_lb_method(stencil=stencil_phase,
                                method='srt',
                                relaxation_rate=w_c,
                                compressible = True,
                                force_model=force_model_h)

allen_cahn_lb = create_lb_update_rule(lb_method=method_phase,
                                      velocity_input = u, 
                                      compressible = True,
                                      optimization = {"symbolic_field": h,
                                                      "symbolic_temporary_field": h_tmp},
                                      kernel_type = 'stream_pull_collide')

allen_cahn_lb.set_main_assignments_from_dict({**allen_cahn_lb.main_assignments_dict, **{C.center: sum_h}})

ast_allen_cahn_lb = ps.create_kernel(allen_cahn_lb, target=dh.default_target, cpu_openmp=True)
kernel_allen_cahn_lb = ast_allen_cahn_lb.compile()
#------------------------------------------------------------------------------------------------------------------------


## collision of g
#force_model = forcemodels.Guo(force_g(0, MRT_collision_op))
method_hydro = create_with_discrete_maxwellian_eq_moments(stencil_hydro, rr_dict, force_model=force_model_g)

hydro_lb_update_rule = get_collision_assignments_hydro(lb_method=method_hydro,
                                                       density=rho,
                                                       velocity_input=u,
                                                       force = force_g,
                                                       optimization={"symbolic_field": g,
                                                                     "symbolic_temporary_field": g_tmp},
                                                       kernel_type='collide_only')

ast_hydro_lb = ps.create_kernel(hydro_lb_update_rule, target=dh.default_target, cpu_openmp=True)
kernel_hydro_lb = ast_hydro_lb.compile()

In [12]:
# periodic Boundarys for g, h and C
periodic_BC_g = dh.synchronization_function(g.name, target=dh.default_target, optimization = {"openmp": True})
periodic_BC_h = dh.synchronization_function(h.name, target=dh.default_target, optimization = {"openmp": True})
periodic_BC_C = dh.synchronization_function(C.name, target=dh.default_target, optimization = {"openmp": True})

# No slip boundary for the phasefield lbm
bh_allen_cahn = LatticeBoltzmannBoundaryHandling(method_phase, dh, 'h',
                                                 target=dh.default_target, name='boundary_handling_h')

# No slip boundary for the velocityfield lbm
bh_hydro = LatticeBoltzmannBoundaryHandling(method_hydro, dh, 'g' ,
                                            target=dh.default_target, name='boundary_handling_g')

wall = NoSlip()
if dimensions == 2:
    bh_allen_cahn.set_boundary(wall, make_slice[:, 0])
    bh_allen_cahn.set_boundary(wall, make_slice[:, -1])

    bh_hydro.set_boundary(wall, make_slice[:, 0])
    bh_hydro.set_boundary(wall, make_slice[:, -1])
else:
    bh_allen_cahn.set_boundary(wall, make_slice[:, 0, :])
    bh_allen_cahn.set_boundary(wall, make_slice[:, -1, :])

    bh_hydro.set_boundary(wall, make_slice[:, 0, :])
    bh_hydro.set_boundary(wall, make_slice[:, -1, :])


bh_allen_cahn.prepare()
bh_hydro.prepare()

In [13]:
ac_g = create_lb_update_rule(stencil = stencil_hydro,
                             optimization={"symbolic_field": g,
                                           "symbolic_temporary_field": g_tmp},
                             kernel_type='stream_pull_only')
ast_g = ps.create_kernel(ac_g, target=dh.default_target, cpu_openmp=True)
stream_g = ast_g.compile()

In [14]:
# definition of the timestep for the immiscible fluids model
def Improved_PhaseField_h_g():
    bh_allen_cahn()
    periodic_BC_h()
    
    dh.run_kernel(kernel_allen_cahn_lb)
    periodic_BC_C()
    dh.run_kernel(kernel_hydro_lb)

    bh_hydro()
    periodic_BC_g()
    
    dh.run_kernel(stream_g)
    dh.swap("h", "h_tmp")
    dh.swap("g", "g_tmp")

In [15]:
Initialize_distributions()

pos_liquid_front = np.zeros((2, tf))
pos_bubble_front = np.zeros((2, tf))
for i in range(0, tf):
    # write out the phasefield
    if gpu:
        dh.to_cpu("C")

    pos_liquid_front[0, i] = i / reference_time
    pos_liquid_front[1, i] = (np.argmax(dh.cpu_arrays["C"][L0//2, :] > 0.5) - Ny//2)/L0

    pos_bubble_front[0, i] = i / reference_time
    pos_bubble_front[1, i] = (np.argmax(dh.cpu_arrays["C"][0, :] > 0.5) - Ny//2)/L0
    
    # do one timestep
    Improved_PhaseField_h_g()

/home/markus/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Numpy has detected that you (may be) writing to an array with
overlapping memory from np.broadcast_arrays. If this is intentional
set the WRITEABLE flag True or make a copy immediately before writing.
  


In [16]:
assert(pos_liquid_front[1, -1] == -0.1953125)
assert(pos_bubble_front[1, -1] == 0.1875)